# **TPS July 2021** ♨

Hello kagglers,

This is the first TPS with time series data. I will be updating this notebook as competition progresses.

***

<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">

<p style="padding: 10px;
              color:white;
          text-align:center;">
Loading requirements
             
</p>
</div>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from sklearn.ensemble import RandomForestRegressor

#### Loading dataset

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv')
sample_submission = pd.read_csv('../input/tabular-playground-series-jul-2021/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
train.info()

we need to convert date_time column to int

#### looking at the statistics

In [ ]:
train.describe()

#### **NOTES**

- There no missing values
- There are NO negative observations
- Feature scaling may be required

<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">

<p style="padding: 10px;
              color:white;
          text-align:center;">
EDA
             
</p>
</div>

Let's visualize the dataset to better understand it.

In [ ]:
carbon_monoxide = train['target_carbon_monoxide']
benzene = train['target_benzene']
nitrogen_oxides = train['target_nitrogen_oxides']

In [ ]:
f = plt.figure(figsize=(15,8))

ax = f.add_subplot(131)
stats.probplot(carbon_monoxide, plot=ax)
ax.set_title('Carbon monoxide probability distribution')

ax1 = f.add_subplot(132)
stats.probplot(benzene, plot=ax1)
ax1.set_title('Benzene probability distribution')

ax2 = f.add_subplot(133)
stats.probplot(nitrogen_oxides, plot=ax2)
ax2.set_title('Nitrogen_oxides probability distribution')

plt.show()

#### **NOTES**

- The target values are NOT Gaussian distributed as the red line does't overlap with the blue points.
- There are EXTREME target observations(Outliers) in the data set.
- There is an OFFSET at 0 for 'benzene' plot.

#### Scatter matrix

In [ ]:
targets = train[['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']].copy()

In [ ]:
pd.plotting.scatter_matrix(targets, alpha=0.5,figsize=(15, 8))
plt.show()

#### **NOTES**

- All the distributions are tail heavy(Right skewed).
- There is a STRONG CORRELATION among target variables
- We can see the OFFSET line in the 'benzene' feature, which we have seen in probability plot. we need to UNDER SAMPLE this data points/remove those to minimize the effect.

<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">

<p style="padding: 10px;
              color:white;
          text-align:center;">
Model
             
</p>
</div>

##### Preprocessing

In [ ]:
train['date_time'] = train['date_time'].astype('datetime64[ns]').astype(np.int64)/10**9
test['date_time'] = test['date_time'].astype('datetime64[ns]').astype(np.int64)/10**9

In [ ]:
train.head()

In [ ]:
carbon_monoxide = carbon_monoxide.values.reshape(-1, 1)
benzene = benzene.values.reshape(-1,1)
nitrogen_oxides = nitrogen_oxides.values.reshape(-1,1)

In [ ]:
X_train = train.drop(['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'], axis=1).copy()

In [ ]:
rf_model = RandomForestRegressor(max_depth=100, max_features='sqrt', min_samples_leaf=3,
                           min_samples_split=12, n_estimators=600,bootstrap=True,random_state=1)
rf_model.fit(X_train, carbon_monoxide)
sample_submission['target_carbon_monoxide'] = rf_model.predict(test)

rf_model = RandomForestRegressor(max_depth=8, max_features='auto', min_samples_leaf=1,
                           min_samples_split=6, n_estimators=1600,bootstrap=True,random_state=1)
rf_model.fit(X_train, benzene)
sample_submission['target_benzene'] = rf_model.predict(test)

rf_model = RandomForestRegressor(max_depth=40, max_features='sqrt', min_samples_leaf=5,
                           min_samples_split=10, n_estimators=800,bootstrap=True,random_state=1)
rf_model.fit(X_train, nitrogen_oxides)
sample_submission['target_nitrogen_oxides'] = rf_model.predict(test)

In [ ]:
sample_submission.to_csv('submission.csv', index=False)